This is an attempt to recreate the parameter estimation [example](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/fig-html/appendix/fig-A-10.html) from James Rawlings book on [Reactor Design](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/) using Pyomo.

In [18]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyomo.environ as pyo
import pyomo.dae as dae

In [19]:
data_df_1 = pd.read_csv("https://raw.githubusercontent.com/notesofdabbler/learn_kipet/master/my_data_sets/ABCD_cb0_1.csv", index_col = 0)
data_df_1.head()

,A,B,C,D
0.000000,1.041989,0.957302,-0.112837,-0.063992
0.544445,0.606033,0.507213,0.335157,0.178739
1.166667,0.408613,0.192682,0.272406,0.249736
1.711111,0.498139,0.171993,0.289555,0.251345
2.333333,0.451026,0.132128,0.196068,0.281246


In [20]:
data_df_2 = pd.read_csv("https://raw.githubusercontent.com/notesofdabbler/learn_kipet/master/my_data_sets/ABCD_cb0_2.csv", index_col = 0)
data_df_2.head()

,A,B,C,D
0.000000,1.003906,1.975654,0.039959,0.009157
0.311111,0.554819,1.383557,0.328321,0.194300
0.666667,0.375822,1.012860,0.272322,0.378839
0.977778,0.305279,0.864239,0.244618,0.496218
1.333333,0.269072,0.667304,0.151982,0.610147


In [21]:
# Convert data to a list of dictionaries
data = [{'A': {k:v for (k, v) in zip(data_df_1.index, data_df_1.A)},
    'C': {k:v for (k, v) in zip(data_df_1.index, data_df_1.C)},
       'init': {'A': 1, 'B': 1, 'C': 0, 'D':0}}]
data.append(
{'B': {k:v for (k, v) in zip(data_df_2.index, data_df_2.B)},
    'D': {k:v for (k, v) in zip(data_df_2.index, data_df_2.D)},
       'init': {'A': 1, 'B': 2, 'C': 0, 'D':0}}
)


In [22]:
def genblock(b, k):
    b.dummy = pyo.Param(initialize = 1)


In [46]:
def ABC_model(m, b, data, disctype, sim):
    
    species = ['A', 'B', 'C', 'D']
    id_species = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
    
    S = np.array([[-1.0, -1.0, 1.0, 0.0],
                  [0.0, -1.0, -1.0, 1.0]
                 ])
    
    meas_t = np.array([list(data[k].keys()) for k in data.keys() if k in species])
    meas_t = list(np.unique(meas_t.flatten()))
    max_t = np.max(meas_t)
           
    c0 = data['init']
    
    def _c_init_rule(b, t, j):
        return c0[j]
    
    b.errsq = pyo.Var(within = pyo.NonNegativeReals)
    b.time = dae.ContinuousSet(bounds = (0.0, max_t), initialize = meas_t)
    b.c = pyo.Var(b.time, species, initialize = _c_init_rule, bounds = (0, 2))
    
    b.dc = dae.DerivativeVar(b.c, wrt = b.time)
    
    def _dcrate(b, t, j):
        
        nrxn = S.shape[0]
        nspec = S.shape[1]
        rrate = {}
        for i in range(nrxn):
            rrate[i] = m.k[i]
            for j2 in range(nspec):
                if S[i, j2] < 0:
                    rrate[i] = rrate[i] * b.c[t, species[j2]]
        
        if t == 0:
            return pyo.Constraint.Skip
        else:
            return b.dc[t, j] == sum(S[i, id_species[j]] * rrate[i] for i in rxns)

    def _dcrate_sim(b, t, j):
        nrxn = S.shape[0]
        nspec = S.shape[1]
        rrate = {}
        for i in range(nrxn):
            rrate[i] = m.k[i]
            for j2 in range(nspec):
                if S[i, j2] < 0:
                    rrate[i] = rrate[i] * b.c[t, species[j2]]
        return b.dc[t, j] == sum(S[i, id_species[j]] * rrate[i] for i in rxns)
    
    if sim == 0:
        b.dcrate = pyo.Constraint(b.time, species, rule = _dcrate)
    else:
        b.dcrate = pyo.Constraint(b.time, species, rule = _dcrate_sim)
    
    for j in species:
        b.c[0, j].fix(c0[j])
    
    def _errsq_rule(b):
        expr = 0
        for j in species:
            if j in data.keys():
                expr = expr + sum((b.c[t, j] - data[j][t]) ** 2 for t in meas_t) 
        return b.errsq == expr 
    b.errsq_cons = pyo.Constraint(rule=_errsq_rule)
    
    if disctype == 'colloc':
        disc = pyo.TransformationFactory('dae.collocation')
        disc.apply_to(b, nfe=20, ncp=2)
    else:
        disc = pyo.TransformationFactory('dae.finite_difference')
        disc.apply_to(b, nfe=500, scheme = 'BACKWARD')
    
    return m

In [47]:
def get_model():
    m = pyo.ConcreteModel()

    rxns = [0, 1]
    m.k = pyo.Var(rxns, initialize = 0.5, bounds = (1e-4, 10))
    
    expts = [0, 1]
    
    def genblock(b, k):
        b.dummy = pyo.Param(initialize = 1)
    m.b = pyo.Block(expts, rule = genblock)
    
    for e in expts:
        m = ABC_model(m, m.b[e], data[e], 'colloc', 0)
    
    m.obj = pyo.Objective(expr = sum(m.b[e].errsq for e in expts), sense = pyo.minimize)
    
    return m

In [48]:
m = get_model()

In [49]:
solver = pyo.SolverFactory('ipopt')
solver.solve(m, tee = True)

Ipopt 3.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:     2914
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      528

Total number of variables............................:      644
                     variables with only lower bounds:        2
                variables with lower and upper bounds:      322
                     variables with only upper bounds:        0
Total number of equality constraints.................:      642
Total number of inequali

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 642, 'Number of variables': 644, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.05825614929199219}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [50]:
# Estimated parameters
m.k[0](), m.k[1](0)

(0.9947794669556348, 2.065614495035512)